# Advent of Code 2021 - Day 12

In [334]:
import java.io.File

sealed interface Cave { val id: String }
@JvmInline value class BigCave(override val id: String) : Cave
@JvmInline value class SmallCave(override val id: String) : Cave

fun String.toCave(): Cave = if (all { it.isUpperCase() }) BigCave(this) else SmallCave(this)

class CaveSystem(block: CaveSystem.() -> Unit) {
    private var _paths: MutableMap<Cave, MutableList<Cave>> = mutableMapOf()
    val paths: Map<Cave, List<Cave>> = _paths

    infix fun String.pathTo(to: String) {
        _paths
            .getOrPut(this.toCave()) { mutableListOf() }
            .let { it.add(to.toCave()) }
    }

    init { block() }
}

val regex = Regex("""(\w+)-(\w+)""")
val caveSystem = File("Day12.input.txt")
    .bufferedReader()
    .readLines()
    .map { regex.find(it)!!.destructured }
    .let {
        CaveSystem { 
            it.forEach { (a, b) -> 
                a pathTo b
                b pathTo a 
            } 
        }
    }

## Part 1

The `caveSystem` defines all `paths` between traversable between `Cave`s. Find all the distinct routes between the "start" and "end" caves that visit `SmallCave`s only once.

In [335]:
fun CaveSystem.routes(
    start: Cave = SmallCave("start"), 
    end: Cave = SmallCave("end"), 
    visited: List<Cave> = listOf(),
    routesFilter: List<Cave>.(List<Cave>) -> List<Cave>,
): List<List<Cave>> {
    fun CaveSystem.routeTo(start: Cave, visited: List<Cave>): List<List<Cave>> =
            if (start == end) listOf(visited)
            else routes(start = start, end = end, visited = visited, routesFilter)

    return if (listOf(start).routesFilter(visited).isEmpty()) listOf()
    else paths[start]
        ?.routesFilter(visited)
        ?.flatMap { routeTo(start = it, visited = visited + start) }
        ?: listOf()
}

caveSystem.routes { visited -> filterNot { it is SmallCave && it in visited } }.size

5212

### Notes

This is another graph traversal problem. The idea is to recursively route to each (valid) connection and track the `visited` route along the way. Once we reach the end we can return it as a distinct route. To enforce that we visit `SmallCave`s only once, we can just check if it has been `visited` and ignore the check for `BigCave`s.

## Part 2

Find all the distinct routes where a single `SmallCave` can be visited twice. The "start" cave can only be visited once.

In [336]:
fun SmallCave.canRoute(visited: List<Cave>): Boolean =
    visited
        .filter { it is SmallCave }
        .groupingBy { it }.eachCount()
        .let { it.getOrDefault(this, 0) < 1 || (it.maxOfOrNull { it.value } ?: 0) < 2 }

caveSystem.routes { visited ->
    filterNot { cave -> (cave is SmallCave && !cave.canRoute(visited)) }
    .filterNot { cave -> (cave == SmallCave("start") && cave in visited) }
}.size

134862

### Notes

This one is kind of annoying and there's probably a more efficient way to do this, but to best reuse the implementation of part 1, we can just change what is a "valid" route. In part 1 `SmallCaves` can only be visited once. I went back to add a `routesFilter` so that way we can add a more complex `SmallCave::canRoute` function. The function just checks whether the current `SmallCave` has only been added once, or if it can be added twice.